# Imports

In [ ]:
import pandas as pd
import numpy as np


# Reading data and Getting insights

In [ ]:
import pandas as pd

df = pd.read_csv('used_cars.csv', encoding='unicode_escape')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
print(f"no of rows -- {df.shape[0]}")
print(f"no of columns -- {df.shape[1]}")

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
type(df.columns)

In [ ]:
df.nunique()

In [ ]:
df['Year'].min()
df['Year'].max()

1998 -- 2019  years

In [ ]:
df['Location'].value_counts()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

# Removing Unwanted Columns

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.drop(['Unnamed: 0', 'New_Price'], axis=1, inplace=True)

In [ ]:
df.isna().sum()

# Converting numerical with units to plain numerical

In [ ]:
df

In [ ]:
df.iloc[0:10, 0:2]

# iloc[rows, columns]

In [ ]:
df.loc[4:50, ["Name", "Mileage", "Kilometers_Driven"]]

In [ ]:
df.head(10)

In [ ]:
df.tail(7)

In [ ]:
type(df.loc[3, "Mileage"])

In [ ]:
df.loc[3, "Mileage"].split()

In [ ]:
type(float(df.loc[3, "Mileage"].split()[0]))

In [ ]:
def clean_mileage(mileage):
  return float(str(mileage).split()[0]) if pd.notna(mileage) else None

def clean_engine(engine):
  return float(str(engine).split()[0]) if pd.notna(engine) else None

def clean_power(power):
  return float(str(power).split()[0]) if pd.notna(power) and power!= "null bhp" else None

In [ ]:
df["Mileage"] = df["Mileage"].apply(clean_mileage)
df["Engine"] = df["Engine"].apply(clean_engine)
df["Power"] = df["Power"].apply(clean_power)

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
na_cols_to_fill = [x for x in df.columns if df[x].isna().sum() != 0]

In [ ]:
na_cols_to_fill

In [ ]:
for i in na_cols_to_fill:
  df[i].fillna(df[i].mean(), inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df

# Label Encoding

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
label_mappings = {}

In [ ]:
cat_cols = [x for x in df.columns if df[x].dtype == "object"]

In [ ]:
cat_cols

In [ ]:
for i in cat_cols:
  df[i] = le.fit_transform(df[i])
  label_mappings[i] = dict(zip(le.classes_, le.transform(le.classes_)))

In [ ]:
label_mappings

In [ ]:
df

In [ ]:
le.classes_

In [ ]:
le.transform(le.classes_)

In [ ]:
df_rm = df.copy()

In [ ]:
cat_cols

In [ ]:
for col in cat_cols:
  reverse_mapping = {v : k for k, v in label_mappings[col].items()}
  df_rm[col] = df[col].map(reverse_mapping)

# Skewness and Kurtosis

Skewness = 0	-- Perfectly symmetrical (e.g., normal distribution)


Skewness > 0	-- Right-skewed (longer tail on the right; more smaller values)

Skewness < 0	-- Left-skewed (longer tail on the left; more larger values)





Kurtosis ≈ 3	-- Mesokurtic – Normal distribution (benchmark)

Kurtosis > 3	-- Leptokurtic – Heavy tails, sharp peak (more outliers)

Kurtosis < 3	-- Platykurtic – Light tails, flat peak (fewer outliers)



In [ ]:
df

In [ ]:
df.skew()

In [ ]:
df.kurt()

To remove skewness and kurtosis we do log transform

In [ ]:
for i in df.columns:
  df[i] = np.log1p(df[i])

In [ ]:
df.skew()

In [ ]:
df.kurt()

# Scaling

In [ ]:
df

In [ ]:
len(df.columns)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
ss = StandardScaler()    # -1 to 1

In [ ]:
ms = MinMaxScaler()      # 0 to 1

In [ ]:
df_scaled = ms.fit_transform(df)

In [ ]:
df_scaled = ss.fit_transform(df)

In [ ]:
df_scaled

In [ ]:
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

In [ ]:
df_scaled

# Removing Outliers

In [ ]:
def remove_outliers(df, column):
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)
  IQR =  Q3-Q1
  lower_bound = Q1-1.5*IQR
  upper_bound = Q3+1.5*IQR

  return df[(df[column] >= lower_bound) & (df[column] <=upper_bound)]


In [ ]:
df_rm = remove_outliers(df_rm, "Kilometers_Driven")

# Visualization

In [ ]:
df_scaled



1.   matplotlib
2.   seaborn
3.   pandas_plotting
4.   plotly



In [ ]:
!pip install matplotlib

: 

In [ ]:
import matplotlib.pyplot as plt

: 

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(20, 20))
cor = df_scaled.corr()
sns.heatmap(cor, cmap= "PiYG", annot=True)
plt.show()

In [ ]:
def correlation(dataset, threshold):
  col_corr = set()
  corr_matrix = dataset.corr()
  for i in range(len(corr_matrix.columns)):
    for j in range(i):
      if (corr_matrix.iloc[i, j]) > threshold:
          colname = corr_matrix.columns[i]
          col_corr.add(colname)
  return col_corr

In [ ]:
var_to_remove = (correlation(df_scaled, 0.75))

In [ ]:
var_to_remove.remove("Price")

In [ ]:
var_to_remove

In [ ]:
# df_new = df_scaled.drop("Power", axis=1)

In [ ]:
plt.scatter(df_rm["Owner_Type"], df_rm["Price"])
plt.xlabel("Owner_type")
plt.ylabel("Price")
plt.show()

In [ ]:
for i in df.columns.drop("Name"):
  plt.scatter(df_rm[i], df_rm["Price"])
  plt.xlabel(f"{i}")
  plt.ylabel("Price")
  plt.xticks(rotation=45)
  plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
scatter_matrix(df_rm, figsize=(20, 20), alpha = 0.8, diagonal="hist")  # diagonal -- kde for density distribution
plt.show()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.pie(df_rm, names="Owner_Type", title="Pie Chart for Owner Type" )
fig.show()

In [ ]:
df_rm["Name"].nunique()

In [ ]:
cat_cols

In [ ]:
pie_cols = cat_cols.copy()
pie_cols.remove("Name")

In [ ]:
pie_cols

In [ ]:
for i in pie_cols:
  fig = px.pie(df_rm, names=i, title=f"Pie Chart for {i}", hole=0.3)
  fig.show()

In [ ]:
num_cols  = [x for x in df_rm.columns if df_rm[x].dtype != "object"]

In [ ]:
num_cols

In [ ]:
fig = px.scatter(df_rm, x="Kilometers_Driven", y="Price", title="KD vs Price", labels={"Kilometers_Driven": "Kilometers Driven", "Price": "Price"}, color = "Price")
fig.show()

In [ ]:
for i in num_cols:
  fig = px.scatter(df_rm, x=i, y="Price", title=f"{i} vs Price", labels={i: i, "Price": "Price"}, color = "Price")
  fig.show()

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sns.boxplot(x="Fuel_Type", y="Price", data=df_rm)
plt.show()

In [ ]:
cat_cols.remove("Name")

In [ ]:
cat_cols

In [ ]:
for i in cat_cols:
  sns.boxplot(x=i, y="Price", data=df_rm)
  plt.xticks(rotation=45)
  plt.show()

In [ ]:
cat_cols

In [ ]:
for i in cat_cols:
  plt.hist(df_rm[i], color="skyblue", edgecolor="Black")
  plt.title(f"Histogram of {i}")
  plt.xticks(rotation=45)
  plt.show()

In [ ]:
num_cols = [x for x in df_rm if df_rm[x].dtype != "object"]

In [ ]:
num_cols

In [ ]:
for i in num_cols:
  plt.figure(figsize=(10, 6))
  sns.kdeplot(data=df_rm[i])
  plt.title(f"KDE Plot of {i}")
  plt.xlabel(f"{i}")
  plt.show()

In [ ]:
for i in df_rm.columns.drop('Name'):
  sns.lineplot(data=df_rm, x=i, y="Price")
  plt.title(f"{i} and Price Distribution")
  plt.xticks(rotation=45)
  plt.show()

# Excel Questions Solved

1. What is the average selling price of used cars by brand?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_g = pd.read_csv("/content/used_cars.csv")

In [ ]:
df_g

In [ ]:
df_g.iloc[0,1].split()[0]

In [ ]:
df_g["Name"]

In [ ]:
df_g["Brand"] = df_g["Name"].apply(lambda x: x.split()[0])

In [ ]:
df_g

In [ ]:
# df_g.groupby("Brand").agg({"Price" : ["mean", "min", "max"]}).reset_index()

In [ ]:
df_g.groupby("Brand").agg({"Price" : ["mean"]}).reset_index()

2. How many cars are available per year?

In [ ]:
q2 = df_g["Year"].value_counts().reset_index()

In [ ]:
q2

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(data=q2, x="Year", y="count")
plt.title("Cars Available Per Year")
plt.xticks(rotation=45)
plt.show()

3. What is the average mileage (kmpl/km/kg) by fuel type?

In [ ]:
df_g

In [ ]:
def clean_mileage(mileage):
  return float(str(mileage).split()[0]) if pd.notna(mileage) else None

def clean_engine(engine):
  return float(str(engine).split()[0]) if pd.notna(engine) else None

def clean_power(power):
  return float(str(power).split()[0]) if pd.notna(power) and power!= "null bhp" else None

In [ ]:
df_g["Mileage"] = df_g["Mileage"].apply(clean_mileage)

In [ ]:
df_g

In [ ]:
q3 = df_g.groupby("Fuel_Type").agg({"Mileage" : "mean"}).reset_index()

In [ ]:
import plotly.express as px

In [ ]:
q3

In [ ]:
fig = px.pie(q3, names="Fuel_Type", values="Mileage")
fig.show()

4. What is the distribution of cars by transmission type?

In [ ]:
# df_g.groupby("Transmission").agg({"Location" : "count"})

In [ ]:
q4 = df_g["Transmission"].value_counts().reset_index()

In [ ]:
fig = px.pie(q4, names = 'Transmission',values="count", title = 'Pie Chart')
fig.show()

5. Which Owner type offers the most cars, and what's the average selling price?

In [ ]:
q5 = df_g.groupby("Owner_Type").agg({"Name":"count", "Price":"mean"}).reset_index()

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 5))

ax1.bar(q5["Owner_Type"], q5["Name"], color='skyblue', label="Car Count")
ax1.set_ylabel("Car Count", color='black')
ax1.tick_params(axis='y', labelcolor='black')

ax2 = ax1.twinx()
ax2.plot(q5["Owner_Type"], q5["Price"], color='red', marker='o', label="Avg Price")
ax2.set_ylabel("Average Price", color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title("Car Count and Average Price by Owner Type")
plt.tight_layout()
plt.show()

6. Which year has the highest average selling price?

In [ ]:
year_price = df_g.groupby("Year").agg({"Price" : "mean"}).reset_index()

In [ ]:
(year_price.sort_values(by="Price", ascending=False)).iloc[0, ]

7. What is the relationship between engine size (binned) and price?

In [ ]:
df_g["Engine"] = df_g["Engine"].apply(clean_engine)

In [ ]:
df_g

In [ ]:
df_g["engine_size_bin"] = pd.cut(df_g["Engine"], bins=[0, 1000, 2000, 3000, 4000, 5000, 6000])

In [ ]:
df_g

In [ ]:
q7 = df_g.groupby("engine_size_bin").agg({"Price" : "mean"}).reset_index()

In [ ]:
q7["engine_size_bin"] = q7["engine_size_bin"].astype(str)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=q7, x="engine_size_bin", y="Price")
plt.title("relationship between engine size and price")
plt.xticks(rotation=45)
plt.show()